In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as *
import boto3
import sys
import re
from datetime import datetime
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1663562373676_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
conf = SparkConf()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark = SparkSession.builder.config(
            conf=conf).enableHiveSupport().getOrCreate()

In [ ]:
DF1 = spark.read.parquet("s3://sjet-datamart-bucket/precog/precog-processed-data/q400/q400128wps/year=2021/month=November/part-00000-4a51bf39-28ee-4a6e-9856-b777f9c41630.c000.snappy.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
DF1.show(5)#createOrReplaceTempView('DF1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+-----------------+-----+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+-----------------+----------+----------+----------------+-----------------+---------------

In [7]:
FirstQuery  = "(select FileName, max(cast(`Radio-Altitude` as int)) as RadioAltitude from Q400Interim group by FileName);"

spark.sql(FirstQuery).createOrReplaceTempView("B")

SecondQuery = "(select a.*, B.RadioAltitude from Q400Interim a inner join B on a.FileName = B.FileName and a.`Radio-Altitude` = B.RadioAltitude)"

spark.sql(SecondQuery).createOrReplaceTempView("C")

ThirdQuery = "(select FileName, RadioAltitude, max(`Time-In-Secs`) as MaxTime, min(`Time-In-Secs`) as MinTime from C group by FileName, RadioAltitude)"

spark.sql(ThirdQuery).createOrReplaceTempView("D")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
FourthQuery = '''(select a.`Time-In-Secs` as `Time-In-Secs-EX`,
                    a.`BARO-CORRECTED-PALT-DERIVED` as `BARO-CORRECTED-PALT-DERIVED-EX`,
                    a.`Radio-Altitude` as `Radio-Altitude-EX`,
                    REPLACE(a.`GLIDE-PATH-DEV-COPILOT-ABOVE`,'*','') as `GLIDE-PATH-DEV-COPILOT-ABOVE-EX`,
                    REPLACE(a.`GLIDE-PATH-DEV-PILOT-ABOVE`,'*','') as `GLIDE-PATH-DEV-PILOT-ABOVE-EX`,
                    'G/SDeviation' as ExceedanceName,
                    case 
                    when Cast(`Time-In-Secs` as int)<D.MinTime Then 'TakeOff'
                    when Cast(`Time-In-Secs` as int) between D.MinTime and D.MaxTime Then 'Airborne'
                    when Cast(`Time-In-Secs` as int)>D.MaxTime Then 'Approach' end Phase,
                     
                    case when
                    case when replace(`GLIDE-PATH-DEV-COPILOT-ABOVE`,'*','') like 'FU%' then 
                    CASE WHEN instr(replace(`GLIDE-PATH-DEV-COPILOT-ABOVE`,'*',''),' ')>0 
                             THEN SUBSTRING(replace(`GLIDE-PATH-DEV-COPILOT-ABOVE`,'*',''),
                             instr(replace(`GLIDE-PATH-DEV-COPILOT-ABOVE`,'*',''),' ')+1,
                             LENGTH(replace(`GLIDE-PATH-DEV-COPILOT-ABOVE`,'*','')))  
                             ELSE NULL END end > 1.00 and 
                    Cast(`Time-In-Secs` as int)>D.MaxTime Then 'High' else 'Low' end TriggerCondition1,
                    
                    case when
                    case when replace(`GLIDE-PATH-DEV-PILOT-ABOVE`,'*','') like 'FU%' then 
                    CASE WHEN instr(replace(`GLIDE-PATH-DEV-PILOT-ABOVE`,'*',''),' ')>0 
                             THEN SUBSTRING(replace(`GLIDE-PATH-DEV-PILOT-ABOVE`,'*',''),
                             instr(replace(`GLIDE-PATH-DEV-PILOT-ABOVE`,'*',''),' ')+1,
                             LENGTH(replace(`GLIDE-PATH-DEV-PILOT-ABOVE`,'*',''))) 
                             ELSE NULL END end > 1.00 and 
                    Cast(`Time-In-Secs` as int)>D.MaxTime Then 'High' else 'Low' end TriggerCondition2,
        
                    a.* from Q400Interim a
                    inner join D  
                    on a.FileName = D.FileName
                    where Cast(`Time-In-Secs` as float)>D.MaxTime and`Main-WOW` = 'Air (0)')'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
spark.sql(FourthQuery).createOrReplaceTempView("E")

FifthQuery = f'''(select * FROM E WHERE TriggerCondition1 = 'High' OR TriggerCondition2 = 'High' and `Radio-Altitude` < '1200' and Phase = 'Approach' )'''

spark.sql(FifthQuery).createOrReplaceTempView("F")

Q400SQLDF4 = spark.sql('''SELECT * FROM F''')

Q400SQLDF4.createOrReplaceTempView("FinalQ400")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
Q400SQLDF5 = spark.sql('''select * from FinalQ400 where dataframename = '128 WPS' and 'GLIDE-PATH-DEV-COPILOT-ABOVE-EX' LIKE '%*%' ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
Q400SQLDF5.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+-------------------------------+-----------------------------+--------------+-----+-----------------+-----------------+--------+-----------------+----------------+----------+--------+------+---------+---+------------+----+------+---------+-------+--------+----+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+---

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+-------------------------------+-----+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+-----------------+----------+----------+----------------+-----------------+-------------------------+-------------------------+----------------------+--------

In [12]:
df3.createOrReplaceTempView('DF3')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df3.select('FileName').distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

271

In [104]:
spark.sql('''SELECT distinct(FileName) 
                FROM DF1 WHERE FileName IN (SELECT distinct FileName FROM DF3) ''').show(100,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+
|FileName                                                    |
+------------------------------------------------------------+
|ALL ac=VT-SUG dep=BOM arr=STV fn=SG 2697 dt=20211230.csv    |
|ALL ac=VT-SUJ dep=JSA arr=JAI fn=SG 3759 dt=20211230.csv    |
|ALL ac=VT-SUG dep=BOM arr=IXY fn=SG 3327 dt=20211219.csv    |
|ALL ac=VT-SUO dep=CCU arr=IXB fn=SG 4027 dt=20211225.csv    |
|ALL ac=VT-SUJ dep=GOP arr=DEL fn=SG 2954 dt=20211225.csv    |
|ALL ac=VT-SUF dep=STV arr=BOM fn=SEJ2696 dt=20211225.csv    |
|ALL ac=VT-SUG dep=BOM arr=IXG fn=SG 3746 dt=20211221.csv    |
|ALL ac=VT-SUG dep=DEL arr=DED fn=SG 2033 dt=20211214.csv    |
|ALL ac=VT-SUX dep=DED arr=DEL fn=SG 2034 dt=20211231.csv    |
|ALL ac=VT-SUQ dep=DEL arr=DED fn=SG 2033 dt=20211224.csv    |
|ALL ac=VT-SUQ dep=DED arr=DEL fn=SG 2034 dt=20211223.csv    |
|ALL ac=VT-SUQ dep=DEL arr=DHM fn=SG 2345 dt=20211228.csv    |
|ALL ac=VT-SUO dep=IXB arr=CCU fn=SG 4028 dt=20211225.c

In [16]:
spark.sql('''SELECT distinct(FileName) FROM DF1 a
                LEFT JOIN DF2 b
                on a.TailNo = b.TailNo AND
                a.FlightNo = b.FlightNo AND
                a.Departure = b.Departure AND
                a.Arrival = b.Arrival AND 
                a.Date = b.Date AND 
                a.LoadedDate = b.MaxLoadDate''').show(500,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------+
|FileName                                                    |
+------------------------------------------------------------+
|ALL ac=VT-SUG dep=BOM arr=IXG fn=SG 3746 dt=20211221.csv    |
|ALL ac=VT-SUG dep=DEL arr=DED fn=SG 2033 dt=20211214.csv    |
|ALL ac=VT-SUX dep=DED arr=DEL fn=SG 2034 dt=20211231.csv    |
|ALL ac=VT-SUQ dep=DEL arr=DED fn=SG 2033 dt=20211224.csv    |
|ALL ac=VT-SUQ dep=DED arr=DEL fn=SG 2034 dt=20211223.csv    |
|ALL ac=VT-SUF dep=STV arr=BOM fn=SEJ2696 dt=20211225.csv    |
|ALL ac=VT-SUG dep=BOM arr=STV fn=SG 2697 dt=20211230.csv    |
|ALL ac=VT-SUJ dep=JSA arr=JAI fn=SG 3759 dt=20211230.csv    |
|ALL ac=VT-SUG dep=BOM arr=IXY fn=SG 3327 dt=20211219.csv    |
|ALL ac=VT-SUO dep=CCU arr=IXB fn=SG 4027 dt=20211225.csv    |
|ALL ac=VT-SUJ dep=GOP arr=DEL fn=SG 2954 dt=20211225.csv    |
|ALL ac=VT-SUQ dep=DEL arr=DHM fn=SG 2345 dt=20211228.csv    |
|ALL ac=VT-SUO dep=IXB arr=CCU fn=SG 4028 dt=20211225.c

In [108]:
spark.sql('''SELECT distinct FileName FROM DF3 where FileName = 'ALL ac=VT-SUX dep=SAG arr=HYD fn=SG 4006 dt=20211225.csv' ''').show(100,truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------+
|FileName                                                |
+--------------------------------------------------------+
|ALL ac=VT-SUX dep=SAG arr=HYD fn=SG 4006 dt=20211225.csv|
+--------------------------------------------------------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
 #ALL ac=VT-SUX dep=HYD arr=SAG fn=SG 4003 dt=20211225 (2).csv

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

262

In [96]:
df2.filter((F.col('TailNo') == 'VT-SUG') & (F.col('FlightNo') == 'SG 3439') & (F.col('Departure') == 'GWL') & (F.col('Arrival') == 'BOM') & (F.col('Date') == '2021-12-25')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+---------+-------+----------+--------------------+
|TailNo|FlightNo|Departure|Arrival|      Date|         MaxLoadDate|
+------+--------+---------+-------+----------+--------------------+
|VT-SUG| SG 3439|      GWL|    BOM|2021-12-25|2022-08-26 05:46:...|
+------+--------+---------+-------+----------+--------------------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
cannot resolve '`FileName`' given input columns: [df2.Arrival, df2.Date, df2.Departure, df2.FlightNo, df2.MaxLoadDate, df2.TailNo]; line 4 pos 18;
'Aggregate ['TailNo, 'FlightNo, 'Departure, 'Arrival, 'Date], ['TailNo, 'FlightNo, 'Departure, 'Arrival, 'Date, 'MAX('LoadedDate) AS MaxLoadDate#874]
+- 'Filter 'FileName LIKE %(_).csv
   +- SubqueryAlias df2
      +- Aggregate [TailNo#10, FlightNo#13, Departure#11, Arrival#12, Date#14], [TailNo#10, FlightNo#13, Departure#11, Arrival#12, Date#14, max(LoadedDate#61) AS MaxLoadDate#803]
         +- Filter NOT FileName#60 LIKE %(_).csv
            +- SubqueryAlias df1
               +- Relation[FlightID#0,DesigFlightNumber#1,FlightNumberARMS#2,FlightDate#3,FromCity#4,ToCity#5,BlockTime#6,ECN#7,EmployeeName#8,RANK#9,TailNo#10,Departure#11,Arrival#12,FlightNo#13,Date#14,Time-In-Secs#15,Time-Per-Sec#16,BARO-CORRECTED-PALT-DERIVED#17,Altitude-BaroL#18,Magnetic-Heading#19,Pitch-Attitude#20,Roll-Attitude#21,Radio-Altitude#22